# Population and employment dynamics in states and counties of the USA

In [ ]:
#nbd voila-hide
import ipywidgets as widgets
import ipyleaflet as leaflet
import matplotlib.pyplot as plt
import json

from popemp import analysis

analysis.prep_data()

In [ ]:
year_selector = widgets.IntRangeSlider(value=(2007, 2012), min=1990, max=2019)
abs_rel_selector = widgets.RadioButtons(options=[('Absolute', 'abs'), ('Relative', 'rel')])
state_selector, _ = analysis.st_cty_selectors()
update_button = widgets.Button(description='Update')
controls = widgets.VBox([year_selector, abs_rel_selector, widgets.Label('Select US or state'), state_selector, update_button])
map_ = analysis.Map()
map_.widget.layout = widgets.Layout(height='100%')
graph = widgets.Output()

In [ ]:
def update_graph(st, cty):
    with graph:
        graph.clear_output(True)
        fig = analysis.plot_growth(st, cty, *year_selector.value)
        fig.set_size_inches(8, 10)
        display(fig)

def update_map(*_):
    map_.upd(state_selector.value, *year_selector.value, abs_rel_selector.value)
    update_graph(state_selector.value, '000')
update_button.on_click(update_map)

def click_area(**kw):
    p = kw['properties']
    update_graph(p['st'], p['cty'])
map_.click_callback = click_area

update_map()

widgets.AppLayout(left_sidebar=controls,
                center=map_.widget, 
                right_sidebar=graph)